In [1]:
import pretty_midi
import os
import numpy as np
import pdb

In [2]:
def get_midi_instruments(midi):
    instruments = midi.instruments
    ins_names = set([ins.name for ins in instruments])
    ins_programs = set([ins.program for ins in instruments])
    return instruments, ins_programs, ins_names

In [3]:
def remove_short_duration_notes(midi):
    instruments = midi.instruments
    for ins in instruments:
        start_end_diff = [note.end - note.start for note in ins.notes]
        lower_threshold = np.percentile(start_end_diff, 10)
        ins_notes = [note for note in ins.notes if note.end - note.start > min(lower_threshold, 0.2)]
        ins.notes = ins_notes
    return midi

In [4]:
def remove_long_duration_notes(midi):
    for ins in midi.instruments:
        ins_notes = sorted(ins.notes, key=lambda k: k.start)
        note_start_dict = {}
        for note in ins_notes:
            if not note.start in note_start_dict:
                note_start_dict[note.start] = []
            note_start_dict[note.start].append(note)
        final_notes = []
        for start_key in note_start_dict:
            start_notes = note_start_dict[start_key]
            if len(start_notes) > 1:
                start_end_diff = [(note, note.end-note.start) for note in start_notes]
                start_end_diff = sorted(start_end_diff, key=lambda k: k[1])
                note_to_add = start_end_diff[0][0]
            else:
                note_to_add = start_notes[0]
            final_notes.append(note_to_add)
        final_notes = sorted(final_notes, key=lambda k: k.start)
        ins.notes = final_notes
    return midi

In [5]:
def make_instrument_monophonic(midi):
    for ins in midi.instruments:
        total_ins_notes = len(ins.notes)
        ins_notes = ins.notes
        final_notes = []
        for i, note in enumerate(ins_notes):
            if (i < (total_ins_notes - 1)) and (note.end > ins_notes[i+1].start):
                note.end = ins_notes[i+1].start
            final_notes.append(note)
        ins.notes = final_notes
    return midi

In [7]:
midi_folder = '../repo_data/indian_raags/'
output_folder = '../repo_data/indian_raags_monophonic/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
for i, midi_file in enumerate(os.listdir(midi_folder)):
    print('({}) {}'.format(i, midi_file))
    midi = pretty_midi.PrettyMIDI(os.path.join(midi_folder, midi_file))
    instruments, ins_programs, ins_names = get_midi_instruments(midi)
    print(instruments)
    midi = remove_short_duration_notes(midi)
    midi = remove_long_duration_notes(midi)
    midi = make_instrument_monophonic(midi)
    midi.write(os.path.join(output_folder, midi_file))

(0) Bhairav.mid
[Instrument(program=104, is_drum=False, name=""), Instrument(program=0, is_drum=True, name="")]
(1) Behag.mid
[Instrument(program=0, is_drum=False, name="untitled")]
(2) Durga.mid
[Instrument(program=0, is_drum=False, name="")]
(3) Des.mid
[Instrument(program=0, is_drum=False, name="")]
(4) bazigar.mid
[Instrument(program=52, is_drum=False, name="Female lead"), Instrument(program=52, is_drum=False, name="Male lead"), Instrument(program=52, is_drum=False, name="Choir Aahs"), Instrument(program=51, is_drum=False, name="Strings/Violins"), Instrument(program=114, is_drum=False, name="Steel Drums"), Instrument(program=112, is_drum=False, name="Tinkle Bell"), Instrument(program=71, is_drum=False, name="Tinkle Bell"), Instrument(program=4, is_drum=False, name="Elec. Piano"), Instrument(program=40, is_drum=False, name="Solo Violin"), Instrument(program=118, is_drum=False, name="Synth Drum"), Instrument(program=55, is_drum=False, name="Orchestra Hit "), Instrument(program=0, is_

In [ ]:
midi = pretty_midi.PrettyMIDI(os.path.join(midi_folder, 'Behag.mid'))
instruments = midi.instruments
instruments

In [ ]:
ins = instruments[0]
sorted(ins.notes, key=lambda k: k.start)